In [6]:
#%pip install scikit-learn==1.2.2


#import sklearn
#print(sklearn.__version__)
#%pip install flask

Note: you may need to restart the kernel to use updated packages.


In [5]:
#Importación de Flask
from flask import Flask, request
import requests

#Importación de módulos para el método inteligente
import pathlib
import pandas as pd
from sklearn.pipeline import Pipeline
from tensorflow.python.keras.models import load_model, model_from_json
from keras import backend as K
import pickle
import keras

app = Flask(__name__)

#Función para cargar preprocesador
def cargarPipeline(nombreArchivo):
    with open(nombreArchivo+'.pickle', 'rb') as handle:
        pipeline = pickle.load(handle)
    print("Preprocesador cargado desde archivo")
    return pipeline

#Función para cargar red neuronal
def cargarNN(nombreArchivo):
    #model = load_model(nombreArchivo+'.h5')   
    model = keras.models.load_model(nombreArchivo+'.h5')
    print("Red Neuronal Cargada desde Archivo") 
    return model

#Función para integrar el preprocesador y la red neuronal en un Pipeline
def cargarModelo():
        #Se carga el Pipeline con el Preprocesador (transformadores)
        nombreArchivoPreprocesador='recursos\pipePreprocesadores1.2'
        pipe=cargarPipeline(nombreArchivoPreprocesador)
        cantidadPasos=len(pipe.steps)
        print("Cantidad de pasos: ",cantidadPasos)
        print(pipe.steps)
        #Se carga la Red Neuronal previamente entrenada
        modeloOptimizado=cargarNN('recursos\modeloRedNeuronalOptimizada')
        #Se agrega la Red Neuronal al final del Pipeline
        pipe.steps.append(['modelNN',modeloOptimizado])
        cantidadPasos=len(pipe.steps)
        print("Cantidad de pasos: ",cantidadPasos)
        print(pipe.steps)
        print('Red Neuronal integrada al Pipeline')
        return pipe

#La siguiente función permite predecir si se aprueba o no un crédito a un nuevo cliente. 
#En la función se define el valor por defecto de las variables, se crea el dataframe con los nuevos valores y
#los nombres de las variables. 
#El método "predict" ejecuta el Pipeline: los pasos de transformación y la clasificación (mediante la red neuronal).
def predecirNuevoCliente(ESTADOCUENTACORRIENTE='A12', PLAZOMESESCREDITO=6, HISTORIALCREDITO='A34', PROPOSITOCREDITO='A43',
                            MONTOCREDITO=1169, SALDOCUENTAAHORROS='A65', TIEMPOACTUALEMPLEO='A75', TASAPAGO=4, 
                            ESTADOCIVILYSEXO='A93', GARANTE='A101', TIEMPORESIDENCIAACTUAL=4, ACTIVOS='A121', EDAD=67, 
                            VIVIENDA='A152', CANTIDADCREDITOSEXISTENTES=2, EMPLEO='A173', CANTIDADPERSONASAMANTENER=2,
                            TRABAJADOREXTRANJERO='A201'):  
    cnames=['ESTADOCUENTACORRIENTE','PLAZOMESESCREDITO','HISTORIALCREDITO','PROPOSITOCREDITO','MONTOCREDITO',
            'SALDOCUENTAAHORROS','TIEMPOACTUALEMPLEO','TASAPAGO','ESTADOCIVILYSEXO','GARANTE','TIEMPORESIDENCIAACTUAL',
            'ACTIVOS','EDAD','VIVIENDA','CANTIDADCREDITOSEXISTENTES','EMPLEO','CANTIDADPERSONASAMANTENER',
            'TRABAJADOREXTRANJERO']
    Xnew=[ESTADOCUENTACORRIENTE,PLAZOMESESCREDITO,HISTORIALCREDITO,PROPOSITOCREDITO,MONTOCREDITO,SALDOCUENTAAHORROS,
          TIEMPOACTUALEMPLEO,TASAPAGO,ESTADOCIVILYSEXO,GARANTE,TIEMPORESIDENCIAACTUAL,ACTIVOS,EDAD,VIVIENDA,
          CANTIDADCREDITOSEXISTENTES,EMPLEO,CANTIDADPERSONASAMANTENER,TRABAJADOREXTRANJERO]
    Xnew_Dataframe = pd.DataFrame(data=[Xnew],columns=cnames)
    pred = (pipe.predict(Xnew_Dataframe) > 0.5).astype("int32")
    pred = pred.flatten()[0]# de 2D a 1D
    if pred==1:
        pred='Crédito Aprobado. Felicidades =)'
    else:
        pred='Crédito Negado. Lo sentimos, intenta en otra ocasión'
    return pred

#Con Shift-Enter se ejecuta el servidor y se inicia cargando el Pipeline (estimador compuesto de preprocesador y red neuronal)
print("********** CARGANDO MODELO: PREPROCESADOR Y RED NEURONAL")
pipe=cargarModelo()

#El servidor queda a la espera de las solicitudes
@app.route("/")
def hello():
    return "Hola Mundo!"

@app.route("/predecirCliente1")
def probarMetodoMLCliente1():
    resul=predecirNuevoCliente(PLAZOMESESCREDITO=10,MONTOCREDITO=40000,TASAPAGO=5,EDAD=25,CANTIDADPERSONASAMANTENER=3,EMPLEO='A173')
    return resul

@app.route("/predecirCliente2")
def probarMetodoMLCliente2():
    resul=predecirNuevoCliente(PLAZOMESESCREDITO=10,MONTOCREDITO=400000,TASAPAGO=5,EDAD=25,CANTIDADPERSONASAMANTENER=3,EMPLEO='A173')
    return resul
        
if __name__ == '__main__':
    app.run()

ModuleNotFoundError: No module named 'sklearn'